In [20]:
import csv
import pandas as pd
import numpy as np

import requests
import re

import json
import time

try: 
    from BeautifulSoup import BeautifulSoup
except ImportError:
    from bs4 import BeautifulSoup

import gender_guesser.detector as gender
d = gender.Detector()

from functions import *
from alive_progress import alive_bar


In [2]:
path = 'C:\\Users\\livia\\Dropbox\\HRP Alliance authorship paper\\Data 2022-04-14\\'
papers_library_wos = 'papers_library_wos.csv'
papers_library = 'papers_library.csv'
papers_pubmed = 'papers_pubmed.csv'

In [3]:
df_papers_library_wos = pd.read_csv(path + papers_library_wos)
df_papers_library = pd.read_csv(path + papers_library)
df_papers_pubmed = pd.read_csv(path + papers_pubmed)

In [4]:
df_papers_library_wos.head(2)

,Key,Item Type,Publication Year,Author,Title,Publication Title,ISBN,ISSN,DOI,Url,...,Programming Language,Version,System,Code,Code Number,Section,Session,Committee,History,Legislative Body
0,TNWHU9ED,journalArticle,1995,"Burrows, T. D.; King, A.; Smith, S. K.; Loke, ...",Human trophoblast adhesion to matrix proteins:...,Molecular Human Reproduction,NaN,NaN,10.1093/molehr/1.7.341,://WOS:000560592900005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,IUVG5AWK,journalArticle,1995,"Jokhi, P. P.; King, A.; Boocock, C.; Loke, Y. W.",Secretion of colony stimulating factor-1 by hu...,Molecular Human Reproduction,NaN,NaN,10.1093/molehr/1.8.414,://WOS:000560593300008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_papers_library.head(2)

,PMID,Title,Authors,Citation,First Author,Journal/Book,Publication Year,Create Date,PMCID,NIHMS ID,DOI
0,34921727,Comparative performance of the human papilloma...,"Hurtado-Salgado E, Cárdenas-Cárdenas L, Salmer...",Int J Cancer. 2022 May 1;150(9):1422-1430. doi...,Hurtado-Salgado E,Int J Cancer,2022,18/12/2021,NaN,NaN,10.1002/ijc.33905
1,34937128,Assessment of the inter-rater agreement of cor...,"Hamzianpour N, Adams VJ, Grundon RA, Linn-Pear...",J Small Anim Pract. 2022 Mar;63(3):188-196. do...,Hamzianpour N,J Small Anim Pract,2022,22/12/2021,NaN,NaN,10.1111/jsap.13462


In [6]:
df_papers_pubmed.head(2)

,PMID,pmid,title,authors_list,citation,journal_book,publication_year,create_date,pmcid,nih_ms,...,fa_country,fa_gender,fa_is_hub,fa_is_stafflast_author,la_country,la_gender,la_is_hub,la_is_staff,is_systematic_review,last_author
0,34921727,NaN,Comparative performance of the human papilloma...,"Erika Hurtado-Salgado, Luz Cárdenas-Cárdenas, ...","Hurtado-Salgado E, Cárdenas-Cárdenas L, Salmer...",International journal of cancer,2022.0,NaN,NaN,NaN,...,NaN,female,NaN,NaN,NaN,male,NaN,NaN,0.0,Eduardo Lazcano-Ponce
1,34937128,NaN,Assessment of the inter-rater agreement of cor...,"N Hamzianpour, V J Adams, R A Grundon, R Linn-...","Hamzianpour N, Adams VJ, Grundon RA, et al. As...",The Journal of small animal practice,2022.0,NaN,NaN,NaN,...,NaN,unknown,NaN,NaN,NaN,unknown,NaN,NaN,0.0,C Heinrich


In [7]:
df_papers_library_wos_not_in_pubmed = df_papers_library_wos[~df_papers_library_wos.DOI.isin(df_papers_pubmed.doi)]

In [19]:
df_papers_library_wos_not_in_pubmed.DOI

0              10.1093/molehr/1.7.341
1              10.1093/molehr/1.8.414
2             10.1093/molehr/2.11.829
3             10.1093/molehr/2.10.725
6       10.1016/j.bpobgyn.2008.08.004
                    ...              
1127          10.9745/ghsp-d-21-00320
1128        10.1136/bmjgh-2020-004512
1129               10.1002/ijgo.13724
1130           10.1055/s-0042-1743094
1131      10.1136/bmjopen-2021-053622
Name: DOI, Length: 649, dtype: object

In [26]:
database = pd.DataFrame()
with alive_bar(len(df_papers_library_wos_not_in_pubmed.DOI), force_tty=True) as bar:
    for doi in df_papers_library_wos_not_in_pubmed.DOI:
        parsed_html = get_parsed_html_from_term(doi)

        pmid = get_pmid(parsed_html)
        
        complete_list_authors = None
        if parsed_html and pmid:
            list_authors = get_authors_list(parsed_html)
            if complete_list_authors is []:
                complete_list_authors = list_authors
            else:
                complete_list_authors = pd.concat([complete_list_authors, list_authors])        

            list_authors = list_authors['full_name']

            database = pd.concat([database, create_row_paper(parsed_html, list_authors)])
        else:
            linha = pd.DataFrame(np.nan, index=[pmid], columns=database.columns)


    

on 0: D:\Google Drive\Analises\HRP\functions.py:133: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.
      
      The code that caused this warning is on line 133 of the file D:\Google Drive\Analises\HRP\functions.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.
      
        parsed_html = BeautifulSoup(html)


|⚠︎                                       | (!) 0/649 [0%] in 31.8s (0.00/s)                                             


TypeError: 'NoneType' object is not subscriptable

In [24]:
doi

'10.1097/01.aids.0000363780.42956.ff'

In [25]:
parsed_html

<!DOCTYPE html>
<html lang="en">
<head itemscope="" itemtype="http://schema.org/WebPage" prefix="og: http://ogp.me/ns#">
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<!-- Mobile properties -->
<meta content="True" name="HandheldFriendly"/>
<meta content="320" name="MobileOptimized"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="https://cdn.ncbi.nlm.nih.gov" rel="preconnect"/>
<link href="https://www.ncbi.nlm.nih.gov" rel="preconnect"/>
<link href="https://www.google-analytics.com" rel="preconnect"/>
<link href="https://cdn.ncbi.nlm.nih.gov/pubmed/4d22484f-0ddb-40f4-9209-86f77a0512e5/CACHE/css/output.35e8b192ea09.css" rel="stylesheet" type="text/css"/>
<link href="https://cdn.ncbi.nlm.nih.gov/pubmed/4d22484f-0ddb-40f4-9209-86f77a0512e5/CACHE/css/output.452c70ce66f7.css" rel="stylesheet" type="text/css"/>
<link href="https://cdn.ncbi.nlm.nih.gov/pubmed/4d22484f-0ddb-40f4-9209-86f77a0512e5/CACHE/css/output.dc65705de2

In [22]:
database
#create_row_paper(parsed_html, list_authors)

,pmid,title,authors_list,citation,journal_book,publication_year,create_date,pmcid,nih_ms,doi,...,fa_country,fa_gender,fa_is_hub,fa_is_stafflast_author,la_country,la_gender,la_is_hub,la_is_staff,is_systematic_review,last_author
23097499,23097499,NOTICE OF RETRACTION: 'MicroRNA 21: response t...,,NOTICE OF RETRACTION: 'MicroRNA 21: response t...,Molecular human reproduction,2012,NaN,PMC3612010,NaN,10.1093/molehr/gas045,...,NaN,unknown,NaN,NaN,NaN,unknown,NaN,NaN,0.0,
23097499,23097499,NOTICE OF RETRACTION: 'MicroRNA 21: response t...,,NOTICE OF RETRACTION: 'MicroRNA 21: response t...,Molecular human reproduction,2012,NaN,PMC3612010,NaN,10.1093/molehr/gas045,...,NaN,unknown,NaN,NaN,NaN,unknown,NaN,NaN,0.0,
9237222,9237222,Activation of elements of the phosphatidylinos...,"B S Houmard, Z Guan, B T Stokes, J S Ottobre","Houmard BS, Guan Z, Stokes BT, Ottobre JS. Act...",Molecular human reproduction,1996,NaN,,NaN,10.1093/molehr/2.11.829,...,NaN,unknown,NaN,NaN,NaN,unknown,NaN,NaN,0.0,J S Ottobre
9239689,9239689,The prolactin inhibition of follicle-stimulati...,"L A Villanueva, I Méndez, S Ampuero, F Larrea","Villanueva LA, Méndez I, Ampuero S, Larrea F. ...",Molecular human reproduction,1996,NaN,,NaN,10.1093/molehr/2.10.725,...,NaN,unknown,NaN,NaN,NaN,unknown,NaN,NaN,0.0,F Larrea
18819848,18819848,Epidemiology of postpartum haemorrhage: a syst...,"Guillermo Carroli, Cristina Cuesta, Edgardo Ab...","Carroli G, Cuesta C, Abalos E, Gulmezoglu AM. ...",Best practice & research. Clinical obstetrics ...,2008,NaN,,NaN,10.1016/j.bpobgyn.2008.08.004,...,NaN,male,NaN,NaN,NaN,unknown,NaN,NaN,1.0,A Metin Gulmezoglu
19035991,19035991,"Evidence-based intrapartum care in Cali, Colom...","A Conde-Agudelo, A Rosas-Bermudez, A M Gülmezoglu","Conde-Agudelo A, Rosas-Bermudez A, Gülmezoglu ...",BJOG : an international journal of obstetrics ...,2008,NaN,,NaN,10.1111/j.1471-0528.2008.01930.x,...,NaN,unknown,NaN,NaN,NaN,unknown,NaN,NaN,0.0,A M Gülmezoglu
18941376,18941376,"HIV, human papillomavirus, and cervical neopla...","Hugo De Vuyst, Flavia Lillo, Nathalie Broutet,...","De Vuyst H, Lillo F, Broutet N, Smith JS. HIV,...",European journal of cancer prevention : the of...,2008,NaN,,NaN,10.1097/CEJ.0b013e3282f75ea1,...,NaN,male,NaN,NaN,NaN,female,NaN,NaN,0.0,Jennifer S Smith
19602214,19602214,Balancing equity and efficiency in health prio...,"Caroline Jehu-Appiah, Rob Baltussen, Charles A...","Jehu-Appiah C, Baltussen R, Acquah C, et al. B...",Value in health : the journal of the Internati...,2008,NaN,,NaN,10.1111/j.1524-4733.2008.00392.x,...,NaN,female,NaN,NaN,NaN,mostly_male,NaN,NaN,0.0,Sam Adjei
18829448,18829448,Expression of epithelial cadherin in the human...,"C I Marín-Briggiler, M F Veiga, M L Matos, M F...","Marín-Briggiler CI, Veiga MF, Matos ML, Echeve...",Molecular human reproduction,2008,NaN,,NaN,10.1093/molehr/gan053,...,NaN,unknown,NaN,NaN,NaN,unknown,NaN,NaN,0.0,M H Vazquez-Levin
19085634,19085634,The effect of calcium supplementation during p...,"Hany Abdel-Aleem, Mario Merialdi, Elwany D Els...","Abdel-Aleem H, Merialdi M, Elsnosy ED, Elsedfy...",The journal of maternal-fetal & neonatal medic...,2009,NaN,,NaN,10.1080/14767050802464569,...,NaN,male,NaN,NaN,NaN,male,NaN,NaN,0.0,Jose Villar


In [10]:
df_papers_library[~df_papers_library.DOI.isin(df_papers_pubmed.doi)]

,PMID,Title,Authors,Citation,First Author,Journal/Book,Publication Year,Create Date,PMCID,NIHMS ID,DOI
79,34594989,Passive Smoking During the Periconceptional Pe...,"Zhang Y, Liu J, Zhang L, Jin L, Greene NDE, Li...",China CDC Wkly. 2021 Sep 10;3(37):778-782. doi...,Zhang Y,China CDC Wkly,2021,01/10/2021,PMC8441180,NaN,10.46234/ccdcw2021.193
